In [1]:
import pandas as pd
import psycopg2
from psycopg2 import sql

# AWS RDS Connection

In [8]:
PGEND_POINT = '' # End_point
PGDATABASE_NAME = 'youtube_test_db' # Database Name example: youtube_test_db
PGUSER_NAME = 'postgres' # UserName
PGPASSWORD = 'asd123qwe' # Password

In [9]:
def connect():
    
    # Set up a connection to the postgres server.
    conn_string = "host="+ PGEND_POINT +" port="+ "5432" +" dbname="+ PGDATABASE_NAME +" user=" + PGUSER_NAME \
                  +" password="+ PGPASSWORD
    
    conn = psycopg2.connect(conn_string)
    print("Connected!")

    # Create a cursor object
    cursor = conn.cursor()
    
    return conn, cursor

In [10]:
conn, cursor = connect()

Connected!


 ## SQL Queries
 

In [11]:
# Creating simple table
query_food = sql.SQL("""CREATE TABLE Menu (
  food_id INT NOT NULL,
  food_name VARCHAR(30) NOT NULL,
  PRIMARY KEY (food_id),
  UNIQUE (food_name)
);""")



In [12]:
cur = conn.cursor()
cur.execute(query_food)
cur.close() # close communication with the PostgreSQL database server
conn.commit() # commit the changes

In [13]:
# insert data into table
query_food_insert = sql.SQL("""
INSERT INTO Menu 
    (food_id, food_name) VALUES (1,'Tacos'), (2,'Tomato Soup'),(3,'Grilled Cheese');
""")

In [14]:
cur = conn.cursor()
cur.execute(query_food_insert)
cur.close() 
conn.commit()

In [15]:
# Query the table

query_menu = sql.SQL("""
SELECT * FROM menu;
""")

In [16]:
cur = conn.cursor()
cur.execute(query_menu)


In [17]:
print(cur.fetchone())

(1, 'Tacos')


In [18]:
cur = conn.cursor()
cur.execute(query_menu)
print(cur.fetchall())

[(1, 'Tacos'), (2, 'Tomato Soup'), (3, 'Grilled Cheese')]


In [19]:
cur = conn.cursor()
cur.execute(query_menu)
rows_to_fetch = 2
print(cur.fetchmany(rows_to_fetch))

[(1, 'Tacos'), (2, 'Tomato Soup')]


In [20]:
cur.close() 
conn.commit()

### DataFrame

In [21]:
cur = conn.cursor()
cur.execute(query_menu)

In [22]:
df_menu = pd.DataFrame(cur.fetchall(), columns=['food_id', 'food_name'])

In [23]:
df_menu

,food_id,food_name
0,1,Tacos
1,2,Tomato Soup
2,3,Grilled Cheese


In [24]:
cur.close() 
conn.commit()

#### Insert dataframe into table

In [25]:
data = {'Name': ['Tom', 'Jerry', 'harry'],
        'Age': [50, 25, 10]}

In [26]:
df_people = pd.DataFrame(data)

In [27]:
df_people

,Name,Age
0,Tom,50
1,Jerry,25
2,harry,10


In [28]:
query_people = sql.SQL("""CREATE TABLE people (
  Name VARCHAR(30) NOT NULL,
  Age INT NOT NULL,
  PRIMARY KEY (Name),
  UNIQUE (Name)
);""")

In [29]:
cur = conn.cursor()
cur.execute(query_people)
cur.close()
conn.commit()

In [30]:
# single insert
def single_inserts(conn, df, table):
    for i in df.index:
        cur = conn.cursor()
        cols  = ','.join(list(df.columns))
        vals  = [df.at[i,col] for col in list(df.columns)]
        query = "INSERT INTO %s(%s) VALUES('%s',%s)" % (table, cols, vals[0], vals[1])
        cur.execute(query)
        cur.close()
        conn.commit()
    print("single insert completed")


In [31]:
single_inserts(conn,df_people,'people')

single insert completed


In [32]:
query_people_table = sql.SQL("""
SELECT * FROM people;
""")

In [33]:
cur = conn.cursor()
cur.execute(query_people_table)
print(cur.fetchall())
cur.close()
conn.commit()

[('Tom', 50), ('Jerry', 25), ('harry', 10)]


In [60]:
# batch insert

In [34]:
apple_df = pd.read_csv('AAPL.csv') # import dataset

In [35]:
apple_df.shape

(1259, 5)

In [36]:
apple_df.sample(5)

,Date,Open,High,Low,Close
482,2019-07-16,51.147499,51.527500,50.875000,51.125000
171,2018-04-19,43.439999,43.847500,43.165001,43.200001
576,2019-11-26,66.735001,66.790001,65.625000,66.072502
1246,2022-07-27,152.580002,157.330002,152.160004,156.789993
281,2018-09-25,54.937500,55.705002,54.924999,55.547501


In [37]:
query_apple = sql.SQL("""CREATE TABLE apple_table (
  Date DATE NOT NULL,
  Open FLOAT,
  High FLOAT,
  Low FLOAT,
  Close FLOAT,
  PRIMARY KEY (Date),
  UNIQUE (Date)
);""")

In [38]:
cur = conn.cursor()
cur.execute(query_apple)
cur.close()
conn.commit()

In [39]:
def update_many(conn, datafrm, table):
    
    # Creating a list of tupples from the dataframe values
    tpls = [tuple(x) for x in datafrm.to_numpy()]
    
    # dataframe columns with Comma-separated
    cols = ','.join(list(datafrm.columns))
    
    # SQL query to execute
    sql = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s,%%s,%%s)" % (table, cols)
    cursor = conn.cursor()
    cursor.executemany(sql, tpls)
    print("Data upload complete successfully...")
    cursor.close()
    conn.commit()

In [40]:
update_many(conn,apple_df,'apple_table')

Data upload complete successfully...


In [41]:
query_apple = sql.SQL("""
SELECT * FROM apple_table;
""")

In [42]:
cur = conn.cursor()
cur.execute(query_apple)
print(cur.fetchall())
cur.close()
conn.commit()

[(datetime.date(2017, 8, 14), 39.830002, 40.052502, 39.6875, 39.962502), (datetime.date(2017, 8, 15), 40.165001, 40.549999, 40.035, 40.400002), (datetime.date(2017, 8, 16), 40.485001, 40.627499, 40.037498, 40.237499), (datetime.date(2017, 8, 17), 40.130001, 40.177502, 39.459999, 39.465), (datetime.date(2017, 8, 18), 39.465, 39.875, 39.18, 39.375), (datetime.date(2017, 8, 21), 39.375, 39.4725, 38.7775, 39.302502), (datetime.date(2017, 8, 22), 39.557499, 40.0, 39.505001, 39.945), (datetime.date(2017, 8, 23), 39.767502, 40.1175, 39.720001, 39.994999), (datetime.date(2017, 8, 24), 40.107498, 40.185001, 39.637501, 39.817501), (datetime.date(2017, 8, 25), 39.912498, 40.139999, 39.817501, 39.965), (datetime.date(2017, 8, 28), 40.035, 40.5, 39.982498, 40.3675), (datetime.date(2017, 8, 29), 40.025002, 40.779999, 40.0, 40.727501), (datetime.date(2017, 8, 30), 40.950001, 40.9725, 40.6525, 40.837502), (datetime.date(2017, 8, 31), 40.91, 41.130001, 40.869999, 41.0), (datetime.date(2017, 9, 1), 41.2

In [43]:
cur = conn.cursor()
cur.execute(query_apple)
df_new_apple = pd.DataFrame(cur.fetchall(), columns=['Date', 'Open','High','Low','Close'])

In [44]:
df_new_apple.head()

,Date,Open,High,Low,Close
0,2017-08-14,39.830002,40.052502,39.687500,39.962502
1,2017-08-15,40.165001,40.549999,40.035000,40.400002
2,2017-08-16,40.485001,40.627499,40.037498,40.237499
3,2017-08-17,40.130001,40.177502,39.459999,39.465000
4,2017-08-18,39.465000,39.875000,39.180000,39.375000


In [45]:
df_new_apple.shape

(1259, 5)

In [46]:
apple_df.shape

(1259, 5)